In [1]:
#run the circuit notebook and the pd/cubase connection notebook
%run circuit.ipynb
#%run pdpython.ipynb
%run Cubpython.ipynb

In [2]:

import pygame #pygame: it is a game engine (it allows me to control screens, midi outputs, etc in a synchronized and easy way)
import numpy as np
from pygame import midi #pygame midi functionality
import threading #threading: it allows me to run two functions at the same time


#initialize everything needed
counter = 0 #it indicates the step of the sequencer
cycle = 0 #it indicates the cycle of the sequencer (cycles of 16 steps each)
pygame.init() 
midi.init()

#set the width and height of the screen [width, height]
size = (700, 500)
screen = pygame.display.set_mode(size)

#set the title of the window
pygame.display.set_caption("My Quantum Computer")

#list of texts to draw on the screen
textsToDraw = []
def log(txt): #function that adds a text to the list of texts to draw
    textsToDraw.append(txt)

#loop until the user clicks the close button
done = False

#FPS (frames per second), which is how fast the screen updates
FPS = 5
clock = pygame.time.Clock()

#config midi. We set by hand the number for the input and output of the Launchpad 
launchpad_id = 2
launchpad_id_out = 5
cubase_id = 9

#show all midi device names
for i in range(midi.get_count()):
    print(midi.get_device_info(i), i)
    if "MIDIIN2" in str(midi.get_device_info(i)[1]):
        launchpad_id = i
    if "MIDIOUT2" in str(midi.get_device_info(i)[1]):
        launchpad_id_out = i

#print("Launchpad input: ", midi.get_device_info(launchpad_id))

#if there is a device named "LPX", connect to it and to Cubase
midiInput = {} 
midiOutput = {}
midiOutputCUB = {}
if("LPX" in str(midi.get_device_info(launchpad_id)[1])):
    log("Connecting...")
    midiInput = midi.Input(launchpad_id)
    midiOutput = midi.Output(launchpad_id_out)
    midiOutputCUB = midi.Output(cubase_id) #cubase is not an interface, so we need a virtual device to prepare the midi notes for cubase
    log("Launchpad & Cubase connected!")

#test cubase
#midiOutputCUB.note_on(70, 127, 5)
#midiOutputCUB.note_on(60, 127, 1)

#There are three types of coordinates: 
# 1. Launchpad coordinates (x,y) = (n1,n2) 
# 2. Step sequencer coordinates (section, step, qubit) 
# 3. Midi coordinate

#function whose input are Launchpad coordinates and returns the MIDI note value
def inverseGetCoordinates(n1, n2): #(n1,n2) = (x,y)
    noteval = n2*10 + n1
    return noteval

#function whose input are Launchpad coordinates that gets the position on the step sequencer 
def getStepPosition(n1, n2):

    #initialize variables
    section = -1 
    step = -1
    qubit = -1

    if n1 > 8 or n2>8: #if out of range (n1 and n2 run from 1 to 8) return -1
        result = -1
    else:
        result = 1
        if n2 > 4:
            section = "up"
            step = n1
            if n2 == 8:
                qubit = 0
            if n2 == 7:
                qubit = 1
            if n2 == 6:
                qubit = 2
            if n2 == 5:
                result = -1

        else:
            section = "down"
            step = n1 + 8
            if n2 == 4:
                qubit = 0
            if n2 == 3:
                qubit = 1
            if n2 == 2:
                qubit = 2
            if n2 == 1:
                result = -1
    return (result, section, step-1,  qubit) #step-1 because the step sequencer runs from 0 to 15 meanwhile the Launchpad coordinates run from 1 to 8

#function whose input are Launchpad coordinates that gets the position of the chords
def getChordsPosition(n1, n2):

    #initialize variables
    result = -1
    section = -1
    step = -1

    #chords are in the fourth (n2 = 5) and eigth (n2 = 1) row of the Launchpad. Thus:
    if (n2==5 or n2==1) and (1<=n1 and n1<=8):
        result = 1
    if n2==5:
        section = "up"
    else:
        section = "down"
    step = n1
    return (result, section, step-1) #step-1 because the step sequencer runs from 0 to 15 meanwhile the Launchpad coordinates run from 1 to 8

#function whose input is the position on the step sequencer that gets the Launchpad coordinates 
def inverseGetStepPosition(qubit, step):
    n1 = step%8 + 1
    n2 = 0
    if step < 8:
        if qubit == 0:
            n2 = 8
        if qubit == 1:
            n2 = 7
        if qubit == 2:
            n2 = 6
    else:
        if qubit == 0:
            n2 = 4
        if qubit == 1:
            n2 = 3
        if qubit == 2:
            n2 = 2
    return n1, n2


#gate labels: 0 = I, 1 = H, 2 = X, 3 = Rup, 4 = Rdown, 5 = CX1, 6 = CX2
gateColors = [17, 25, 33, 41, 49, 57, 58] #colors for the gates (in the Launchpad)

#variables to control the gates/measures switches and the bass/chords ones (mode selection)  
gatemes = 0
bassch = 0

#function that shows/renders the circuit in the leds of the Launchpad. It receives the step as an input
def lights(step):
    for qubit in range(3):
        n1, n2 = inverseGetStepPosition(qubit, step)
        noteval = inverseGetCoordinates(n1, n2)
        gateNum = MG[step][qubit]
        noteVel = gateColors[gateNum]

        if gatemes==1: #if we are in the measure mode
             #gates in yellow (noteVel = 13) means activated in the Launchpad
             #gates in red (noteVel = 1) means not activated in the Launchpad
            if MM[step][qubit]:
                noteVel = 13
            else:
                noteVel = 1

        if step==counter: #light that iterates through the steps (noteVel = 5) to indicate the current step
            noteVel = 5

        midiOutput.note_on(noteval, noteVel) #send the midi note to the Launchpad
    return 

#lights function for bass/chords
def lights2():
    if bassch==0: #chords mode
        for i in range(2):
            for j in range(8): #1 off/ 84 on
                n1 = j + 1
                n2 = 5 if (i==0) else 1
                noteVal = inverseGetCoordinates(n1, n2)
                noteVel = 84 if (ch[i][j]) else 1  #ch is a boolean array that indicates which chord notes are activated
                midiOutput.note_on(noteVal, noteVel)
    else: 
        for i in range(2): #bass mode
            for j in range(8): #1 off/ 4 on
                n1 = j + 1
                n2 = 5 if (i==0) else 1
                noteVal = inverseGetCoordinates(n1, n2)
                noteVel = 4 if (bass[i]==j) else 1 #bass is an integer array that indicates which bass note is activated
                midiOutput.note_on(noteVal, noteVel)
    return 


#function that updates a circuit from the circuit list based on MG
def updateCircuit(step):
    changeCircuit(MG[step], step)
    
#function that shifts the quantum gates
#gate labels: 0 = I, 1 = H, 2 = X, 3 = Rup, 4 = Rdown, 5 = CX1, 6 = CX2
def shiftGate(qubit, step):
    global MG #call the global variable MatrixGates

    #if you have created a CX gate and your press the shift button, the CX gate is deleted
    if MG[step][qubit]==5 or MG[step][qubit]==6:
       for i in range(3):
           if MG[step][i]==5 or MG[step][i]==6:
            MG[step][i]=0 #delete the CX gate = setting the identity gate

    #else, shift the gate +1 (modulo 5)
    else: 
        MG[step][qubit]= (MG[step][qubit] + 1)%5

    updateCircuit(step)
    return

#function that implements a CX gate
def cxGate(qubit1, qubit2, step):
    global MG
    MG[step][qubit1] = 5
    MG[step][qubit2] = 6
    updateCircuit(step)
    return

#variables used in the function handleStepInput below
noteBuffer = [0,0] #[qubit, step]
state = 0

#this function controles the mode changes and each mode behaviour (modes:measure/gate)
def handleStepInput(stepPosition): #stepPosition = (result, section, step, qubit)
    global state
    global MM
    global MG
    global noteBuffer

    if stepPosition[0]==-1: #check the input is valid
        return
    step = stepPosition[2]
    qubit = stepPosition[3]

    if gatemes==1: #measure mode
        changeMeasure(qubit, step) #toggle for the measure mode (on/off)  
        
    else: #gate mode 
        #state machine
        if state==0: #state: just shifting a gate
            state = 1
            shiftGate(qubit, step)
            noteBuffer = [qubit, step] #memory of the qubit and step where the shift was made to create a CX gate

        elif state==1: #state: when a key is held to create a CX gate
            state = 0
            if step == noteBuffer[1]:
                cxGate(qubit, noteBuffer[0], step) #create a CX gate

#this function controles the mode changes and each mode behaviour (modes:chords/bass)
ch = [[False for i in range(8)] for i in range(2)] #chords array 
bass =[0,0] #bass array
def handleChordsInput(stepPosition): #stepPosition = (result, section, step, qubit)
    if stepPosition[0]==-1:
        return
    if bassch==0: #chords mode
        if stepPosition[1] == "up":
            ch[0][stepPosition[2]] = not ch[0][stepPosition[2]]
        else:
            ch[1][stepPosition[2]] = not ch[1][stepPosition[2]]
    else: #bass mode
        if stepPosition[1] == "up":
            bass[0] = stepPosition[2]
        else:
            bass[1] = stepPosition[2]

    return

#function that updates the quantum state
def updateState():
    global counter
    global quantumState
    quantumState = quantumState.evolve(circuits[counter])

    #extract
    (rA, rM1, rM2) = extract(MM[counter][0], MM[counter][1], MM[counter][2])
    
    return (rA, rM1, rM2)


"""
#funcion that maps the music to the annotations/measures
def handleMusic(rA, rM1, rM2):
    #To do
    #print(f"rA: {rA}, rM1: {rM1}, rM2: {rM2}")
    return
"""

#function that runs the step-sequencer. It actualizes the quantum state and the counter and cycle variables
def quantumAlgorithmStep():
    global counter
    global cycle
    (rA, rM1, rM2) = updateState()
    #handleMusic(rA, rM1, rM2)
    counter = (counter + 1)%16
    if(counter==0):
        cycle = (cycle + 1)%4 #number of cycles
        #thread with function apiRequest
        #threading.Thread(target=apiRequest).start()
    return (rA, rM1, rM2)

#loop that turns off the lights of the Launchpad at the beginning
for n1 in np.arange(1, 8+1):
    for n2 in np.arange(1, 8+1):
        noteval = inverseGetCoordinates(n1, n2)
        val = 0
        midiOutput.note_on(noteval, val)

"""
#code that turns the Launchpad into a chess board (just for fun)
for n1 in np.arange(1, 8+1):
    for n2 in np.arange(1, 8+1):
        noteval = inverseGetCoordinates(n1, n2)
        val = int(((n1+n2)%2) * 127)
        midiOutput.note_on(noteval, val)
"""     

#array that stores which buttons are pressed (just to keep track of the state of the buttons)
isPressed = [False for i in range(127)]

# -------- Main Program Loop -----------
#Parts:
#1. Events: midi events and pygame events.
#2. Logic: quantum algorithm.
#3. Send data to PureData.
#4. Drawing: lights and console.


while not done: #MAIN LOOP STARTS HERE
    # --- This section handles the MIDI input events
    if midiInput.poll(): #if there are any midi events in the stack that are yet unsolved
                         #poll() checks if there are pending tasks in the stack
        midi_events = midiInput.read(10) #if there are events left, then read them

        midi_evs = pygame.midi.midis2events(midi_events, midiInput.device_id) #process them to convert the MIDI events into pygame events

        for m_e in midi_evs:#now we have to process the events one by one iterating through the list with a for loop

            if m_e.type == pygame.midi.MIDIIN:#if the MIDI type of the event is "note"

                #note values
                noteVal = m_e.data1 #note value from 0 to 127
                noteVel = m_e.data2 #note velocity from 0 to 127

                #mode buttons: a mode is activated if its button has been pressed (vel = 127) 
                               # and deactivated if its button has been released (vel = 0)
                if noteVal == 19: #button to change the mode (measure/gate)
                    if noteVel == 127: #this buttons on the right column are not sensitive to the velocity so 127 is the only possible value
                        gatemes = 1
                    else:
                        gatemes = 0
                
                if noteVal == 29: #button to change the mode (chords/bass)
                    if noteVel == 127:
                        bassch = 1
                    else:
                        bassch = 0
                    
                if noteVal == 39: #button to transpose one semitone down
                    if noteVel == 127:
                        offset2 = offset2 - 1 #-1 is one semitone down in midi notes

                if noteVal == 49: #button to transpose one semitone up
                    if noteVel == 127:
                        offset2 = offset2 + 1 #+1 is one semitone up in midi notes

                #get the coordinates from the midi note value  
                def getCoordinates(noteval): #function that gets the coordinates from the MIDI note value
                    strNote = str(noteval) #function that gets the coordinates from the note value separating the digits 
                    return (int(strNote[1]), int(strNote[0]))    
                (n1, n2) = getCoordinates(noteVal)
                
                #get the position on the step sequencer and the chords
                (result, section, step, qubit) = getStepPosition(n1, n2)
                (result2, section2, step2) = getChordsPosition(n1, n2)
                #if result is -1 within these functions, the coordinates do not belong to those areas of the Launchpad
                #Example: coordinates of the fourth row belong to Chords. Thus, the first function would return -1 and the second one would return 1

                if noteVel>0 and not isPressed[noteVal]: #if a button is pressed and it was not pressed before, then this happens (we do not want anything to happen when unpressing or when holding)
                    handleStepInput((result, section, step, qubit)) #this function handles the gates/measures input depending on the mode activated (gatemes variable)
                    handleChordsInput((result2, section2, step2)) #this function handles the chords/bass input depending on the mode activated (bassch variable)
                    isPressed[noteVal] = True #this variable ensures that a note is only processed once when it is pressed, to avoid repetitions of the same event
                
                else: #if a note is unpressed, the CX gate is deleted (state = 0) and the isPressed variable is set to False
                    state = 0
                    isPressed[noteVal] = False

    #handle pygame events (the ones that are not MIDI events)
    #the only event we are handling in here is the closing of the window console
    for event in pygame.event.get(): #user did something
        if event.type == pygame.QUIT: #if user clicked close
            done = True #flag that we are done so we exit this loop, since the main loop only works when done is False

    #we finished with the events part. Now we get into the logic of the program

    # --- Game logic should go here
    (_rA, _rM1, _rM2) = quantumAlgorithmStep() #evolves the quantum state, updates the counter and cycle variables and extracts the measures

    # --- Send data to PureData
    #What we need to send: chord, bass, melody, snare, hihat, counter, cycle
    #prepare the data to send to PureData
    chordData = [[ch[0][i] for i in range(8)], [ch[1][i] for i in range(8)]]
    bassData = [bass[0], bass[1]]
    melodyData = _rA
    snareData = _rM1
    #print(_rM1)
    hihatData = _rM2
    #print(_rM2)
    counterData = counter
    cycleData = cycle

    #send the music information to PureData
    sendMusic(chordData, bassData, melodyData, snareData, hihatData, counterData, cycleData) #function in pdpython.ipynb

    # --- Drawing code should go here

    #light colours
    for i in range(16):
        lights(i) #update the lights of the Launchpad of the step i 
    lights2() #handles the lights of the chords/bass

    #Fill the screen with this colour -> RGB: 1, 128, 109
    screen.fill((1, 128, 109))

    #draw textsToDraw list of texts. The last text should be at the bottom
    lastTxt = len(textsToDraw) - 10 #we want to print the last 10 texts from teh list textsToDraw
    #if it has 5000 items we print from 4990 to 5000.
    lastTxt = max(0, lastTxt) #if there are less than 10 texts to print, we erase the negative elements

    for i in range(lastTxt,len(textsToDraw)): #print the last 10 texts
        #get text
        text = textsToDraw[i]
        #size of text
        font = pygame.font.Font(None, 36)
        #color of text
        text = font.render(text, 1, (200, 200, 200))
        #position of text
        textpos = text.get_rect()
        textpos.centerx = screen.get_rect().centerx
        height = screen.get_rect().height
        textpos.centery = height - 50 - (len(textsToDraw)-i) * 50
        #draw text
        screen.blit(text, textpos)

    #Additionally we draw on a corner the mode we are at (gate/mes only)
    text = str(gatemes)
    font = pygame.font.Font(None, 36)
    text = font.render(text, 1, (200, 200, 200))
    textpos = text.get_rect()
    textpos.centerx = screen.get_rect().width - 50
    textpos.centery = 50
    screen.blit(text, textpos)
    
    #print variables data (counterdata etc) on screen using log
    #log(f"counter: {counterData}")
    #log(f"cycle: {cycleData}")
    #log(f"bass: {bassData}")
    #log(f"chords: {chordData}")
    #log(f"melody: {melodyData}")
    #log(f"snare: {snareData}")
    #log(f"hihat: {hihatData}")


    #go ahead and update the screen with what we've drawn.
    pygame.display.flip()

    #limit to frames per second
    clock.tick(FPS)

#OUT OF THE MAIN LOOP
#close the window and quit.
pygame.quit()
#quit midi (midi handler initialized at the beginning)
midi.quit()



pygame 2.2.0 (SDL 2.0.22, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
(b'MMSystem', b'Microsoft MIDI Mapper', 0, 1, 0) 0
(b'MMSystem', b'Focusrite USB MIDI', 1, 0, 0) 1
(b'MMSystem', b'LPX MIDI', 1, 0, 0) 2
(b'MMSystem', b'MIDIIN2 (LPX MIDI)', 1, 0, 0) 3
(b'MMSystem', b'loopMIDI Port', 1, 0, 0) 4
(b'MMSystem', b'Microsoft GS Wavetable Synth', 0, 1, 0) 5
(b'MMSystem', b'Focusrite USB MIDI', 0, 1, 0) 6
(b'MMSystem', b'LPX MIDI', 0, 1, 0) 7
(b'MMSystem', b'MIDIOUT2 (LPX MIDI)', 0, 1, 0) 8
(b'MMSystem', b'loopMIDI Port', 0, 1, 0) 9
